## <center>Scientific Programming - 7MRI0020 - 2021/2022</center>


## <center>Week 07 - Advanced Python - Part 01</center>


### <center>School of Biomedical Engineering & Imaging Sciences</center>
### <center>King's College London</center>

# Exercises
This purpose of this notebook is to practice and explore concepts with decorators.

### Exercise 1:
Create a function which squares the input of the function it decorates and halves its return value.

In [11]:
def square_and_half(func):
    """
    Wraps the callable `func`, which can take one argument.
    """
    # your code here
    def something(v):
        return func(v**2)/2
    return something

@square_and_half
def add5(v):
    return v + 5

print(add5(4))  # equivalent to (4**2+5)/2 

10.5


In [10]:
# for understanding

def square_and_half(func):
    def something(v):
        return func(v**2)/2
    return something

@square_and_half
def add5(v):
    return v + 5

# add5 = square_and_half(add5)

print(add5(4))

10.5


### Exercise 2:
`square_and_half` only accepts the single argument for `func`. Let's improve it by allows any number of arguments to be passed to `func`. Remember the syntax for positional `*` and keyword `**` arguments.

In [16]:
def square_and_half(func):
    """
    Wraps the callable `func`, which can take any number of arguments.
    """
    # your code here
    def something(*args,**kwargs):
        return func(*args, **kwargs)
    return something

@square_and_half
def myadd(a, b):
    return a + b

@square_and_half
def mysum(*values):
    return sum(values)

print(myadd(5, 4))
print(mysum(1,2,3,4,5))

9
15


### Exercise 3:
One useful application of decorators is as a caching mechanism for some function implementing an algorithm, also called memoization. For given inputs, the decorated function's output is stored, subsequent calls will return this stored result rather than calling the function again. 

Implement this concept in a simple way. What are the technical concerns that should be kept in mind for this construct?

In [18]:
cacheDict = dict()
def cache(func):
    """
    Caches the output from `func` for given inputs. 
    """
    # your code here
    def something(*args):
        if args in cacheDict:
            return cacheDict[args]
        else:
            cacheDict[args] = func(*args)
            return cacheDict[args]
    return something
        


def power(a,b):
    return (a**b)+-(b**a)


%timeit power(5,7)


@cache
def power(a,b):
    return (a**b)+-(b**a)


%timeit power(5,7)

print(power(5,3) == ((5**3)+-(3**5)))

686 ns ± 28.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
174 ns ± 3.23 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
True


A more obvious demonstration of the value of caching is with a recursive function. The Fibonacci sequence can be defined as follows:

In [19]:
def fib(a):
    if a in (0,1):
        return a
    else:
        return fib(a-1)+fib(a-2)

This will involve significant duplicate calculation. Consider `fib(4)`, this will call `fib(3)` and `fib(2)`. However `fib(3)` will also call `fib(2)` as well as `fib(1)`. Obviously caching the values for each call will eliminate duplicate computation.

In [20]:
# painfully slow
%timeit fib(30)

@cache
def fib_cached(a):
    if a in (0,1):
        return a
    else:
        return fib_cached(a-1) + fib_cached(a-2)

# much better
%timeit fib_cached(30)
    
print(fib(30),fib_cached(30))

293 ms ± 15.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
166 ns ± 5.58 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
832040 832040


### Exercise 4:
**Hard Version**: implement a cache which can be initialized with a set of pre-computed cache values. This would look something like this:

In [23]:
precache={
    0:0,
    1:1,
    2:1,
    3:2,
    4:3,
    5:5,
    6:8
}

def cache_init(precache):
    def someFunc(func):
        def something(*args):
            if args in precache:
                return precache[args]
            else:
                precache[args] = func(*args)
                return precache[args]
        return something
    return someFunc
        
    
@cache_init(precache)
def fib_cached(a):
    if a in (0,1):
        return a
    else:
        return fib_cached(a-1) + fib_cached(a-2)
    
%timeit fib_cached(30)
print(fib_cached(30))

157 ns ± 0.852 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
832040


Note that the expression `cache(precache)` is evaluated to the actual decorator applied to the decorated function. If this is implemented as a function it will have to return the decorator itself, as a class it gets even more interesting.